<a href="https://colab.research.google.com/github/Mongeese1/phyiscs-big-data/blob/master/hhhhhh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as math
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy as sp
import scipy.special as special
from scipy.misc import derivative
import pandas as pd
import sympy as sym
from scipy.optimize import fmin
import inspect as ins
e=math.e
pi=math.pi

In [78]:
G = 6.67*10**(-11)    # (m**3)*(kg**-1)*(s**-2)
c = 3*10**8    # m*(s**-1)

m1 = 10**30 # kg
m2 = 10**30 # kg

M = (m1+m2) # kg
n = m1*m2/(M**2) #dimensionless
M_chirp = n**(3/5)*M # kg
t_c = 100# secs
phi_c = 100 #rads



alpha = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
PN = 2 

SNR = 10

fs = 40 # Hz
f0 = 150 # Hz
s0 = 9**(-46) #Hz**-1

def S(y,S_O,indef):
  if indef==0:
    y = f/f0
  else:
    pass
  S = s0*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S

# y = sym.symbols('y')
# indefa = sym.integrate((f**(-7/6))/S(f,s0,1),y)

# defa = float(indefa.replace(y,f0) - indefa.replace(y,fs))

# A = np.sqrt(SNR / 4 * defa)

def compute_A(SNR, dx):
  #perform integral using trapezoidal sums
  accumulator = 0
  for i in range (int(F_low/dx), int(F_high/dx - 1)):
   x = i*dx
   fi = (x**(-7/6) / (S(x,s0,1)))
   fi2 = ((x + dx)**(-7/6) / (S(x + dx,s0,1)))
   accumulator += dx * (fi + fi2)/2
  # divide SNR by integral multiplied by 4
  A = math.sqrt(SNR/(4*accumulator))
  
  print ("A = ", A, "\n")
  return A

A = compute_A(10,0.01)


# defa = float(indefa.replace(y,f0) - indefa.replace(y,fs))
print(A)
print(defa)
F_high = (6**(3/2)*pi*M*G*(c**-3))**-1 
F_low = fs

var_dict = [t_c,phi_c,M_chirp,n]
 
def waveform(f,A,variables,PN,alpha):
  M = M_chirp*n**(-3/5)
  v = (pi*M*f*G/c**3)**(1/3) 
  PN = PN*2
  zum = np.sum(alpha[0:PN])

  psi = 2*pi*f*t_c - phi_c - pi/4 + 3/(128*n*v**5)*zum

  return A * f**(-7/6) * e**(1j * psi)

origwave = waveform(f,A,var_dict,2,alpha)

#accuracy indicates how accurate deriv should be, input powers of 10, bigger numbers take longer
def deriv(f,A,variables,num,accuracy):
  # if num == 0:
  dx = float(variables[num])/accuracy
  dparams = variables.copy()
  dparams[num] += dx
  dy = waveform(f,A,dparams,1,alpha) - waveform(f,A,var_dict,1,alpha)
  # print(dy)
  return dy/dx
def generate_matrix(A, variables,dx):
  n = len(variables)
  matrix = np.empty([n,n]) 
  # dx = variables[str(num)]/accuracy
  
# Start filling the matrix
  for i in range (n):
    for j in range (n):
      
      #start integration (again with trap sums) using eq 2.11 of the paper as a reference instead of the plug and play
      accumulator = 0
      for k in range(int(F_low/dx), int(F_high/dx-1)):
        x1 = k*dx
        part_i1 = deriv(x1, A, variables,i,10**5)
        part_j1 = deriv(x1, A, variables,j,10**5)
        f1 = ((part_i1 * np.conj(part_j1))+(np.conj(part_i1) * part_j1))/S(x1,s0,1)  
        
        x2 = k*dx + dx 
        part_i2 = deriv(x2, A, variables,i,10**5)
        part_j2 = deriv(x2, A, variables,j,10**5)
        f2 = ((part_i2 * np.conj(part_j2))+(np.conj(part_i2) * part_j2))/S(x2,s0,1)
        
        accumulator += np.real(dx * (f1 + f2)/2)
      
      #once integration is done add it to the matrix
      matrix[i][j] = accumulator*2
  print("Fischer matrix : \n",  matrix)
  return matrix


for i in range(4):
  matrix = generate_matrix(A,var_dict,1)
 
print(matrix)


A =  1302.5566133171903 

1302.5566133171903
45997.72789063209
Fischer matrix : 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Fischer matrix : 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Fischer matrix : 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Fischer matrix : 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
